## Normal and Multivariate Normal in R

The thing that makes R very popular is the vast number of opensource packages that is aviable, and easily installed.
We will start with installing a couplte of packages that we will use:

In [ ]:
#install.packages("NHANES") # NHANES is survey data collected by the US National Center for Health Statistics

Once the package is installed you can active it by using:

In [ ]:
library(NHANES)
#help(NHANES)

In [ ]:
head(NHANES) #as NHANES ia a data set we can print it

We will now collect the data used in the first lecture:

In [ ]:
index <- NHANES$Age > 20 # pick all rows where age is greater then 20.
# the data contains NA (not a number which is annoying so lets remove them also)
index <- (NHANES$Age > 20) & (rowSums(is.na(NHANES[,c('Weight','Height')]))==0) 
data <- NHANES[index, c("Weight", "Height")] # now removed all data that has NA for either Weight or Height
head(data)

Let us now explore the Height data and fit a Normal distribution to it.

In [ ]:
options(repr.plot.width=4, repr.plot.height=3) #adjust figure size for 
hist(data$Height, breaks = 80, main=" histogram Height", xlab='Height',freq=F)

The Normal distribution has two parameter: the mean $\mu$ and standard devation $\sigma$, thus they need to be fitted from the data:

In [ ]:
mu    = mean(data$Height)
sigma = sd(data$Height)

Now lets examin the fit to the data by comparing the normal density 
$$f(\cdot; \mu,\sigma)= \frac{1}{2\pi \sigma} \exp\left(-\frac{1}{2\sigma^2} (\cdot - \mu) \right)^2$$ 
to the histogram:
1. The function seq gives a grid on which we evaulate the density on.
2. The normal density can be caculated using function dnorm.
3. Finally to draw the lines on top of the histogram we use line rather then plot

In [ ]:
grid <- seq(min(data$Height), max(data$Height),length=2000) 
f <- dnorm(grid, mean= mu, sd = sigma) #f(Y|\theta) (Y- is vector, \theta - value )results in a vecctor!
hist(data$Height, 
     breaks = 80, 
     main=" histogram Height", 
     xlab='Height',freq=F)
lines(grid, 
      f,
      col='red',
      lwd = 2)

The fit is ok, but not perfect as one can see that the normal density, has to much mass near the mean.

The probability that a person is between 160 to 180 cm can be caculated as follows:

In [ ]:
mean(data$Height > 160 & data$Height < 180)

Let us compare this to this if we used the fitted Normal distribution instead:

Recall that 
$$\mathbb{P}( 160 < X < 180;\mu, \sigma) = \mathbb{P}(X < 180;\mu, \sigma) - \mathbb{P}(X < 160;\mu, \sigma)= \int_{160}^{180} f(x;\mu, \sigma) dx$$

In [ ]:
x = seq(160,180, length = 2000)  
dx = (180-160)/2000
f <- dnorm(x, mean= mu, sd = sigma)
trapz <- sum(dx * f)
print(paste(" trapetz method = ", trapz, sep = "")) # formula (4.2) in the book

f_int <- integrate(function(x){dnorm(x, mean = mu, sd = sigma)},
          lower=160,
         upper = 180)[1]
print(paste(" R integration  = ", f_int, sep = "")) # using R integrate
P180 = pnorm(180, mean = mu, sd = sigma)
P160 = pnorm(160, mean = mu, sd = sigma)
print(paste(" pnorm          = ", P180-P160, sep = "")) # pnorm is very usefull

 # Going into two dimensions 
 

In [ ]:
#install.packages("mvtnorm")
library(mvtnorm)

In [ ]:
rho <- 0
sigma_x <- 1
sigma_y <- 1
mu  <- c(0,0)
Sigma <- matrix(c(sigma_x^2,  rho * sigma_x * sigma_y, rho * sigma_x * sigma_y,sigma_y^2),nrow=2,ncol=2)
print(Sigma)
XY <- rmvnorm(100, mean = mu, sigma = Sigma)
plot(XY[,1],XY[,2], 
     xlab='x',
     ylab='y', 
     pch = 20)
hist(XY[,1],50,xlab='x',ylab='f_X()',main='')
hist(XY[,2],50,xlab='y',ylab='f_Y()',main='')

We will now study height and weight simultaneously. First we make a 2D histogram of the data. A very nice package for plotting is [ggplot2](http://ggplot2.org/); it has a steep learning curve but is very flexible.

In [ ]:
#install.packages("ggplot2")

In [ ]:
library(ggplot2)
library(RColorBrewer)
rf <- colorRampPalette(rev(brewer.pal(11,'Spectral')))
r <- rf(32)

In [ ]:
p <- ggplot(data, aes(Height, Weight))
p <- p + stat_bin2d() + stat_bin2d(bins=20) + scale_fill_gradientn(colours=r)
p <- p + ylab('Weight')
print(p)

Here one can see if one is a bit experience one can note that the some weights are to extreme to fit a normal distribution. 

In [ ]:
Sigma <- cov( data)
mu    <- colMeans(data)
#install.packages("ellipse")
library(ellipse)
ellipse_data <- ellipse(Sigma, centre = mu, level = 0.9)
ellipse_data <- data.frame(Weight = ellipse_data[,"Weight"],
                           Height = ellipse_data[,"Height"])
p <- p + geom_path(aes( x= Height, y = Weight), data = ellipse_data, size=1.5, colour='red')

print( p)

In this case one can instead transform the data.

In [ ]:
data$Weight <- log(data$Weight)
p <- ggplot(data, aes(Height, Weight))
p <- p + stat_bin2d() + stat_bin2d(bins=20) + scale_fill_gradientn(colours=r)
p <- p + ylab('log(Weight)')
Sigma <- cov( data)
mu    <- colMeans(data)
ellipse_data <- ellipse(Sigma, centre = mu, level = 0.9)
ellipse_data <- data.frame(Weight = ellipse_data[,"Weight"],
                           Height = ellipse_data[,"Height"])
p <- p + geom_path(aes( x= Height, y = Weight), data = ellipse_data, size=1.5, colour='red')

print( p)

## Conditional distributions

Now we want to predict the weight of a person that is 170 cm long, assuming log(weight) and height have multivariate Normal distribution. 


If we treat the height as indpendent $\left(f_{X,Y}(a,b)=f_X(a)f_Y(b) \right)$, then the best we can do is use $f_X(\cdot):$

In [ ]:
grid <- seq(min(data$Weight), log(150),length=2000) 
f <- dnorm(grid, mean= mu[1], sd = sqrt(Sigma[1, 1]))
plot(grid, 
      f,
     type= 'l',
      col='red',
      lwd = 1.5,
      xlab = 'log(weight)')

If we instead use the multivariate distribution, we can instead use the conditional distribution:
$$
f_{X|Y}(\cdot| Y = 180),
$$
Which can be done in R as follows:

In [ ]:
install.packages("condMVNorm")
library(condMVNorm)

In [ ]:
res <- condMVN(X.given       = 180,   # the value of the known index (190cm)
               mean          = mu,    # the mean parameter
               sigma         = Sigma, # the Covariance matrix
               dependent.ind = 1,     # which index we want to generate the density for  (Weight)
               given.ind     = 2)     # which index is known  (Height)
f_cond <- dnorm(grid, 
           mean= res$condMean, 
           sd = res$condVar)
plot(grid, 
     f_cond,
     type= 'l',
     col='red',
     lwd = 1.5,
     xlab = 'log(weight)')

Putting the two densities in the same figure, transforming the scale.

In [ ]:
plot(exp(grid), 
     f,
     type= 'l',
     col='red',
     lwd = 1.5,
     xlab = 'weight',
     ylim = c(0, max(c(f, f_cond))))
lines(exp(grid), 
     f_cond,
     col='blue',
     lwd = 1.5)